In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex客户端库：用于批量预测的自定义训练图像分类模型，附带解释

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_image_classification_batch_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_image_classification_batch_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示如何使用Python的Vertex客户端库来训练和部署自定义图像分类模型，以进行批量预测并提供解释。

数据集

本教程使用的数据集是[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/cifar10)中的[CIFAR10数据集](https://www.tensorflow.org/datasets/catalog/overview)。您将使用的数据集版本已内置于TensorFlow中。训练模型预测图像属于十个类别中的哪一类：飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船、卡车。

### 目标

在本教程中，您将使用 Vertex 客户端库从 Python 脚本中创建一个带有训练流程的自定义模型，并在 Google 预构建的 Docker 容器中执行批处理预测和解释。您也可以选择使用 `gcloud` 命令行工具或在线使用 Google Cloud 控制台创建自定义模型。

执行的步骤包括：

- 创建一个用于训练模型的 Vertex 自定义作业。
- 训练 TensorFlow 模型。
- 检索并加载模型工件。
- 查看模型评估。
- 设置解释参数。
- 将模型上传为 Vertex `Model` 资源。
- 进行带解释的批处理预测。

### 成本

本教程使用Google Cloud（GCP）的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage价格](https://cloud.google.com/storage/pricing)，并使用[Pricing计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成成本估算。

安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新版本的 *google-cloud-storage* 库。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install -U tensorflow

In [ ]:
if os.environ["IS_TESTING"]:
    ! pip3 install -U opencv-python

### 重新启动内核

一旦您已经安装了Vertex客户端库和Google*云存储*，您需要重新启动笔记本内核以便它可以找到这些包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果您有这个选项，确保在 GPU 运行时中运行此笔记本。在 Colab 中，选择* **Runtime > Change Runtime Type > GPU**

### 设置您的 Google Cloud 项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个 Google Cloud 项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得 300 美元的免费信用用于计算/存储成本。

2. [确保为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex API 和 Compute Engine API。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已安装在 Google Cloud Notebook 中。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 在此笔记本中的所有命令中使用正确的项目。

**注意**: Jupyter 以 `!` 为前缀运行带有 `$` 前缀的 Python 变量的行作为 shell 命令，并将这些命令插入到这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改“REGION”变量，该变量用于笔记本的其余部分操作。以下是支持Vertex的区域。我们建议您选择距离最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您不能使用多区域存储桶进行Vertex训练。并非所有区域都支持所有Vertex服务。有关最新的每个区域的支持，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行实时教程会话，您可能会使用共享的测试帐户或项目。为了避免资源创建的用户之间的名称冲突，您为每个实例会话创建一个时间戳，并附加到将在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google Cloud帐户

**如果您正在使用Google Cloud笔记本**，则您的环境已经经过身份验证。跳过此步骤。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证通过oAuth。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**单击创建服务帐户**。

在**服务帐户名称**字段中输入名称，然后单击**创建**。

在**授予此服务帐户访问项目的权限**部分，单击“Role”下拉列表。在过滤框中输入“Vertex”，然后选择**Vertex管理员**。在过滤框中输入“Storage Object Admin”，然后选择**Storage Object Admin**。

单击创建。包含您的密钥的JSON文件将下载到您的本地环境中。

在下面的单元格中将您的服务帐户密钥路径作为GOOGLE_APPLICATION_CREDENTIALS变量输入，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论你使用什么笔记本环境，以下步骤都是必须的。**

当你使用 Vertex 客户端库提交一个自定义训练作业时，你需要将包含训练代码的 Python 包上传到一个云存储桶中。Vertex 会从这个包中运行代码。在本教程中，Vertex 也会在同一个桶中保存由你的作业产生的训练模型。随后，你可以基于此输出创建一个 `Endpoint` 资源，以便提供在线预测。

请设置你的云存储桶的名称。存储桶的名称必须在所有 Google Cloud 项目中是全局唯一的，包括你的组织之外的那些项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最终，通过检查其内容验证对云存储桶的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

导入Vertex客户端库

将Vertex客户端库导入到我们的Python环境中。

In [ ]:
import time

import google.cloud.aiplatform_v1beta1 as aip
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

#### 顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：顶点 API 服务的数据集、模型、作业、流水线和端点服务端点。
- `PARENT`：顶点数据集、模型、作业、流水线和端点资源的位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

硬件加速器

设置用于训练和预测的硬件加速器（例如 GPU）。

设置变量 `TRAIN_GPU / TRAIN_NGPU` 和 `DEPLOY_GPU / DEPLOY_NGPU`，以使用支持 GPU 的容器映像和分配给虚拟机实例的 GPU 数量。例如，为了在每个 VM 上分配 4 个 Nvidia Telsa K80 GPU 使用 GPU 容器映像，您应该指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

对于 GPU，可用的加速器包括：
   - aip.AcceleratorType.NVIDIA_TESLA_K80
   - aip.AcceleratorType.NVIDIA_TESLA_P100
   - aip.AcceleratorType.NVIDIA_TESLA_P4
   - aip.AcceleratorType.NVIDIA_TESLA_T4
   - aip.AcceleratorType.NVIDIA_TESLA_V100

否则，请指定 `(None, None)` 以使用一个在 CPU 上运行的容器映像。

*注意*：在 TF 2.3 之前的 TF 版本中，有关 GPU 支持的自定义模型将无法在本教程中加载。这是一个已知问题，在 TF 2.3 中已修复 -- 这是由于在 serving 函数中生成静态图操作导致的。如果在您自己的自定义模型中遇到此问题，请使用带有 GPU 支持的 TF 2.3 的容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

#### 容器（Docker）镜像

接下来，我们将为训练和预测设置 Docker 容器镜像

- TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/training/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.1-15:latest`
- TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-1:latest`
- TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-2:latest`
- TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-3:latest`
- TensorFlow 2.4
   - `gcr.io/cloud-aiplatform/training/tf-cpu.2-4:latest`
   - `gcr.io/cloud-aiplatform/training/tf-gpu.2-4:latest`
- XGBoost
   - `gcr.io/cloud-aiplatform/training/xgboost-cpu.1-1`
- Scikit-learn
   - `gcr.io/cloud-aiplatform/training/scikit-learn-cpu.0-23:latest`
- Pytorch
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-4:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-5:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-6:latest`
   - `gcr.io/cloud-aiplatform/training/pytorch-cpu.1-7:latest`

最新列表请参阅 [用于训练的预建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers).

- TensorFlow 1.15
   - `gcr.io/cloud-aiplatform/prediction/tf-cpu.1-15:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf-gpu.1-15:latest`
- TensorFlow 2.1
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-1:latest`
- TensorFlow 2.2
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-2:latest`
- TensorFlow 2.3
   - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-3:latest`
   - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-3:latest`
- XGBoost
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-2:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-1:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-90:latest`
   - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-82:latest`
- Scikit-learn
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest`
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-22:latest` 
   - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-20:latest`

最新列表请参阅 [用于预测的预建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

TRAIN_IMAGE = "gcr.io/cloud-aiplatform/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量 `TRAIN_COMPUTE` 和 `DEPLOY_COMPUTE` 来配置您将用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个 vCPU 3.75GB 内存。
     - `n1-highmem`：每个 vCPU 6.5GB 内存
     - `n1-highcpu`：每个 vCPU 0.9 GB 内存
 - `vCPUs`：数量为 \[2, 4, 8, 16, 32, 64, 96 \]

*注意：以下类型不支持用于训练：*

 - `standard`：2个 vCPUs
 - `highcpu`：2、4 和 8 个 vCPUs

*注意：您还可以使用 n2 和 e2 机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

教程

现在您已经准备好开始创建自己的定制模型并为CIFAR10进行训练。

## 建立客户端

Vertex客户端库采用客户端/服务器模型。在您的一侧（Python脚本），您将创建一个客户端，该客户端将从Vertex服务器发送请求并接收响应。

在本教程中，您将针对工作流程中的不同步骤使用不同的客户端。因此请提前设置它们。

- 用于`Model`资源的Model服务。
- 用于部署的Endpoint服务。
- 用于批处理作业和自定义训练的Job服务。
- 用于提供服务的Prediction服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["job"] = create_job_client()
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

## 训练模型

您可以使用容器镜像的两种方式来训练定制模型：

- **使用Google Cloud预构建容器**。如果您使用预构建容器，您还需指定一个要安装到容器镜像中的Python包。这个Python包包含了用于训练定制模型的代码。

- **使用您自己的定制容器镜像**。如果您使用自己的容器，容器需要包含您用于训练定制模型的代码。

## 准备您的自定义作业规范

现在您的客户已经准备好了，您的第一步是为您的自定义培训作业创建一个作业规范。作业规范将包括以下内容：

- `worker_pool_spec`：用于训练的机器类型和数量的规范（单台或分布式）
- `python_package_spec`：预先构建容器中要安装的Python软件包的规范。

准备机器规格

现在为您的自定义训练作业定义机器规格。这告诉 Vertex 需要为训练提供哪种类型的机器实例。
- `machine_type`：要提供的 GCP 实例类型，例如 n1-standard-8。
- `accelerator_type`：硬件加速器的类型，如果有的话。在本教程中，如果之前设置了变量 `TRAIN_GPU != None`，则使用 GPU；否则将使用 CPU。
- `accelerator_count`：加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

### 准备您的磁盘规格

（可选）现在为您的自定义训练作业定义磁盘规格。这将告诉 Vertex 在每台机器实例中为训练提供什么类型和大小的磁盘。

- `boot_disk_type`：SSD 或 Standard。SSD 速度更快，Standard 价格更便宜。默认为 SSD。
- `boot_disk_size_gb`：磁盘大小（GB）。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

### 定义工作池规范

接下来，您将为自定义训练作业定义工作池规范。工作池规范将包括以下内容：

- `replica_count`：要提供的此机器类型的实例数量。
- `machine_spec`：硬件规范。
- `disk_spec`：（可选）磁盘存储规范。

- `python_package`：要安装在虚拟机实例上的Python训练包，以及要调用的Python模块，以及Python模块的命令行参数。

现在让我们更深入地了解Python包规范：

- `executor_image_spec`：这是为您的自定义训练作业配置的Docker镜像。

- `package_uris`：这是要安装在提供的实例上的python训练包的位置（URIs）列表。这些位置需要在一个云存储桶中。这些可以是单独的python文件或整个包的zip（归档）文件。在后一种情况下，作业服务将解压缩（解档）内容到docker镜像中。

- `python_module`：要调用运行自定义训练作业的Python模块（脚本）。在本例中，您将调用`trainer.task.py` —— 注意，无需添加`.py`后缀。

- `args`：传递给相应Python模块的命令行参数。在本例中，您将设置：
   - `"--model-dir=" + MODEL_DIR`：存储模型工件的云存储位置。有两种方法告诉训练脚本要保存模型工件的位置：
      - 直接：将云存储位置作为命令行参数传递给您的训练脚本（设置变量`DIRECT = True`），
      - 间接：服务将云存储位置作为环境变量`AIP_MODEL_DIR`传递给您的训练脚本（设置变量`DIRECT = False`）。在这种情况下，您在作业规范中告诉服务模型工件位置。
   - `"--epochs=" + EPOCHS`：训练的时期数量。
   - `"--steps=" + STEPS`：每个时期的步骤（批次）数量。
   - `"--distribute=" + TRAIN_STRATEGY"`：用于单个或分布式训练的训练分布策略。
      - `"single"`：单个设备。
      - `"mirror"`：单台计算实例上的所有GPU设备。
      - `"multi"`：所有计算实例上的所有GPU设备。

In [ ]:
JOB_NAME = "custom_job_" + TIMESTAMP
MODEL_DIR = "{}/{}".format(BUCKET_NAME, JOB_NAME)

if not TRAIN_NGPU or TRAIN_NGPU < 2:
    TRAIN_STRATEGY = "single"
else:
    TRAIN_STRATEGY = "mirror"

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
        "--distribute=" + TRAIN_STRATEGY,
    ]

worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "disk_spec": disk_spec,
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [BUCKET_NAME + "/trainer_cifar10.tar.gz"],
            "python_module": "trainer.task",
            "args": CMDARGS,
        },
    }
]

### 组装工作规范

现在组装完整的自定义工作规范描述：

- `display_name`：您分配给此自定义工作的可读名称。
- `job_spec`：自定义工作的规范。
    - `worker_pool_specs`：机器VM实例的规范。
    - `base_output_directory`：这告诉服务在何处保存模型产品 (当变量 `DIRECT = False` 时) 的云存储位置。然后，服务会将位置传递给训练脚本作为环境变量 `AIP_MODEL_DIR`，路径将采用以下形式：

                <output_uri_prefix>/model

In [ ]:
if DIRECT:
    job_spec = {"worker_pool_specs": worker_pool_spec}
else:
    job_spec = {
        "worker_pool_specs": worker_pool_spec,
        "base_output_directory": {"output_uri_prefix": MODEL_DIR},
    }

custom_job = {"display_name": JOB_NAME, "job_spec": job_spec}

### 检查培训包

#### 包布局

在开始培训之前，您将查看Python包如何组装以进行自定义培训工作。在解压缩后，该包包含以下目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`是将包安装到Docker镜像的操作环境中的说明。

文件`trainer/task.py`是用于执行自定义培训工作的Python脚本。*请注意*，在我们在工作人员池规范中引用它时，我们将目录斜杠替换为点(`trainer.task`)并删除文件后缀(`.py`)。

#### 包装组合

在接下来的步骤中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: CIFAR10 image classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py 文件内容

在下一个单元格中，您将编写培训脚本 task.py 的内容。我们不会详细讨论，只是让您浏览一下。总之：

- 从命令行获取要保存模型工件的目录（`--model_dir`），如果未指定，则从环境变量 `AIP_MODEL_DIR` 中获取。
- 从 TF Datasets（tfds）加载 CIFAR10 数据集。
- 使用 TF.Keras 模型 API 构建模型。
- 编译模型（`compile()`）。
- 根据参数 `args.distribute` 设置训练分发策略。
- 使用参数 `args.epochs` 和 `args.steps` 训练模型（`fit()`）。
- 将训练好的模型保存（`save(args.model_dir)`）到指定的模型目录中。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for CIFAR-10

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv("AIP_MODEL_DIR"), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.01, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64


def make_datasets_unbatched():

  # Scaling CIFAR10 data from (0, 255] to (0., 1.]
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label


  datasets, info = tfds.load(name='cifar10',
                            with_info=True,
                            as_supervised=True)
  return datasets['train'].map(scale).cache().shuffle(BUFFER_SIZE).repeat()


# Build the Keras model
def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=args.lr),
      metrics=['accuracy'])
  return model


# Train the model
NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS
train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_cnn_model()

model.fit(x=train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(args.model_dir)

把培训脚本存储到您的云存储桶中

然后，您将培训文件夹打包成一个压缩的tar球，然后存储到您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_cifar10.tar.gz

### 训练模型

现在开始在Vertex上训练您的自定义训练作业。使用此辅助函数`create_custom_job`，该函数接受以下参数：

-`custom_job`：自定义作业的规范。

辅助函数调用作业客户端服务的`create_custom_job`方法，并使用以下参数：

-`parent`：指向`Dataset`、`Model`和`Endpoint`资源的Vertex位置路径。
-`custom_job`：自定义作业的规范。

您将展示`response`对象中返回的一些字段，最感兴趣的是其中的两个：

`response.name`：分配给此自定义训练作业的Vertex完全限定标识符。您需要保存此标识符以在后续步骤中使用。

`response.state`：自定义训练作业的当前状态。

In [ ]:
def create_custom_job(custom_job):
    response = clients["job"].create_custom_job(parent=PARENT, custom_job=custom_job)
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = create_custom_job(custom_job)

现在获取您创建的自定义作业的唯一标识符。

In [ ]:
# The full unique ID for the custom job
job_id = response.name
# The short numeric ID for the custom job
job_short_id = job_id.split("/")[-1]

print(job_id)

获取自定义作业的信息

接下来，使用这个辅助函数`get_custom_job`，它接受以下参数：

- `name`：自定义作业的Vertex完全合格的标识符。

这个辅助函数调用作业客户端服务的`get_custom_job`方法，参数如下：

- `name`：自定义作业的Vertex完全合格的标识符。

如果您还记得，在调用`create_custom_job`方法时，您在`response.name`字段中获得了自定义作业的Vertex完全合格的标识符，并将该标识符保存在变量`job_id`中。

In [ ]:
def get_custom_job(name, silent=False):
    response = clients["job"].get_custom_job(name=name)
    if silent:
        return response

    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = get_custom_job(job_id)

部署

训练上述模型可能需要超过20分钟的时间。

一旦您的模型训练完成，您可以通过将`end_time`减去`start_time`来计算训练模型所花费的实际时间。对于您的模型，我们需要知道保存模型的位置，这是Python脚本保存在您的本地云存储桶中的`MODEL_DIR + '/saved_model.pb'`。

In [ ]:
while True:
    response = get_custom_job(job_id, True)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_path_to_deploy = None
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        if not DIRECT:
            MODEL_DIR = MODEL_DIR + "/model"
        model_path_to_deploy = MODEL_DIR
        print("Training Time:", response.update_time - response.create_time)
        break
    time.sleep(60)

print("model_to_deploy:", model_path_to_deploy)

加载保存的模型

您的模型以 TensorFlow SavedModel 格式存储在 Cloud Storage 存储桶中。现在从 Cloud Storage 存储桶中加载它，然后您可以做一些事情，如评估模型和做预测。

要加载，您可以使用 TF.Keras 的 `model.load_model()` 方法，传递保存模型的 Cloud Storage 路径 -- 由 `MODEL_DIR` 指定。

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在找出模型的表现如何。

### 加载评估数据

您将使用`tf.keras.datasets`中的`load_data()`方法加载CIFAR10测试（保留）数据。这将返回一个包含两个元素的元组数据集。第一个元素是训练数据，第二个是测试数据。每个元素也是一个包含两个元素的元组：图像数据和相应的标签。

您不需要训练数据，这就是为什么我们将其加载为`(_, _)`。

在您运行数据进行评估之前，您需要对其进行预处理：

x_test:
1. 通过将每个像素除以255来对像素数据进行归一化（重新缩放）。这将用0到1之间的32位浮点数替换每个单字节整数像素。

y_test：
2. 标签目前是标量（稀疏）的。如果您回顾一下`trainer/task.py`脚本中的`compile()`步骤，您将发现它是为稀疏标签而编译的。因此，我们无需再做任何处理。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import cifar10

(_, _), (x_test, y_test) = cifar10.load_data()
x_test = (x_test / 255.0).astype(np.float32)

print(x_test.shape, y_test.shape)

### 进行模型评估

现在评估自定义作业中模型的效果如何。

In [ ]:
model.evaluate(x_test, y_test)

## 上传模型以供服务

接下来，您将把您的 TF.Keras 模型从自定义作业上传到 Vertex `Model` 服务，这将为您的自定义模型创建一个 Vertex `Model` 资源。在上传过程中，您需要定义一个服务函数，将数据转换为模型所期望的格式。如果您将编码数据发送到 Vertex，您的服务函数将确保数据在传递到模型作为输入之前在模型服务器上进行解码。

### 服务函数如何工作

当您向在线预测服务器发送请求时，请求将被一个 HTTP 服务器接收。HTTP 服务器从 HTTP 请求内容主体中提取预测请求。提取的预测请求将被转发到服务函数。对于 Google 预构建的预测容器，请求内容将作为 `tf.string` 传递给服务函数。

服务函数由两部分组成：

- `预处理函数`:
  - 将输入 (`tf.string`) 转换为底层模型的输入形状和数据类型（动态图）。
  - 执行与训练底层模型期间相同的数据预处理 -- 例如，归一化，缩放等。
- `后处理函数`:
  - 将模型输出转换为接收应用程序期望的格式 -- 例如，压缩输出。
  - 为接收应用程序打包输出 -- 例如，添加标题，创建 JSON 对象等。

预处理和后处理函数都被转换为与模型融合的静态图。来自底层模型的输出被传递给后处理函数。后处理函数将转换/打包后的输出传递回 HTTP 服务器。HTTP 服务器将输出作为 HTTP 响应内容返回。

在为 TF.Keras 模型构建服务函数时需要考虑的一个问题是，它们作为静态图运行。这意味着您不能使用需要动态图的 TF 图操作。如果您这样做，您将在服务函数的编译过程中收到错误提示，指出您正在使用不受支持的 EagerTensor。

### 图像数据的服务功能

为了将图像传递给预测服务，您需要将压缩（例如，JPEG）图像字节编码为 base 64 格式 -- 这样可以使内容在通过网络传输二进制数据时安全保护免受修改。由于部署的模型期望输入数据为原始（未压缩）字节，因此您需要确保将 base 64 编码的数据转换回原始字节，然后将其作为输入传递给部署的模型。

为解决此问题，需要定义一个服务函数(`serving_fn`)，并将其附加到模型作为预处理步骤。添加`@tf.function`装饰器，以便服务函数与底层模型融合在一起（而不是在 CPU 上游处理）。

当您发送预测或解释请求时，请求内容将被 base 64 解码为一个 Tensorflow 字符串 (`tf.string`)，然后传递给服务函数(`serving_fn`)。服务函数会将`tf.string`预处理为原始（未压缩）的 numpy 字节(`preprocess_fn`)，以匹配模型的输入要求:
- `io.decode_jpeg`- 解压缩 JPG 图像，返回具有三个通道（RGB）的Tensorflow张量。
- `image.convert_image_dtype` - 将整数像素值更改为 32 位浮点数，并重新缩放像素数据在 0 和 1 之间。
- `image.resize` - 调整图像大小以匹配模型的输入形状。

此时，数据可以传递给模型(`m_call`)。

#### XAI 签名

当将服务函数与底层模型保存回来时（`tf.saved_model.save`），您需要将服务函数的输入层指定为` serving_default` 签名。

对于 XAI 图像模型，您需要保存来自服务函数的两个附加签名:

- `xai_preprocess`: 服务函数中的预处理函数。
- `xai_model`: 调用模型的具体函数。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(32, 32))
    return resized


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 32, 32, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(
    model,
    model_path_to_deploy,
    signatures={
        "serving_default": serving_fn,
        # Required for XAI
        "xai_preprocess": preprocess_fn,
        "xai_model": m_call,
    },
)

## 获取serving函数的签名

您可以通过将模型重新加载到内存中，并查询每个层对应的签名，来获取模型的输入和输出层的签名。

在进行预测请求时，您需要将请求路由到serving函数而不是模型，因此您需要知道serving函数的输入层名称--这将在稍后进行预测请求时使用。

您还需要知道serving函数的输入和输出层的名称，以构建解释元数据--这将在随后讨论。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)
serving_output = list(loaded.signatures["serving_default"].structured_outputs.keys())[0]
print("Serving function output:", serving_output)

input_name = model.input.name
print("Model input name:", input_name)
output_name = model.output.name
print("Model output name:", output_name)

### 解释规范

在进行预测时要获得解释，您必须启用解释能力并在将自定义模型上传到 Vertex `Model` 资源时设置相应的设置。这些设置称为解释元数据，包括：

- `parameters`：这是用于解释您的模型的可解释性算法的规范。您可以在以下选项之间进行选择：
  - Shapley - *注意*，不推荐用于图像数据 -- 可能非常耗时
  - XRAI
  - Integrated Gradients
- `metadata`：这是算法在自定义模型上应用的规范。

#### 解释参数

让我们深入了解可解释性算法的设置。

#### Shapley

为每个特征分配结果的贡献，并考虑特征的不同排列。该方法提供了对精确 Shapley 值的采样近似。

使用案例：
  - 对表格数据进行分类和回归。

参数：

- `path_count`：这是算法处理的特征路径数量。精确 Shapley 值的近似值需要 M！条路径，其中 M 是特征数量。对于 CIFAR10 数据集，这将是 784（28*28）。

对于任何非平凡数量的特征，这将非常耗时。您可以将特征路径数量减少到 M * `path_count`。

#### Integrated Gradients

一种基于梯度的方法，可高效计算具有与 Shapley 值相同公理性质的特征归因。

使用案例：
  - 对表格数据进行分类和回归。
  - 对图像数据进行分类。

参数：

- `step_count`：这是用于近似剩余总和的步骤数。步骤越多，积分逼近越准确。通常的经验法则是50步，但随着步数的增加，计算时间也会增加。

#### XRAI

基于综合梯度方法，XRAI 评估图像的重叠区域创建一个突显相关区域而不是像素的显著性图，来创建一个突显相关区域的显著性图。

使用案例：

  - 对图像数据进行分类。

参数：

- `step_count`：这是用于近似剩余总和的步骤数。步骤越多，积分逼近越准确。通常的经验法则是50步，但随着步数的增加，计算时间也会增加。

在下一个代码单元格中，将变量 `XAI` 设置为您将在自定义模型上使用的解释算法。

In [ ]:
XAI = "ig"  # [ shapley, ig, xrai ]

if XAI == "shapley":
    PARAMETERS = {"sampled_shapley_attribution": {"path_count": 10}}
elif XAI == "ig":
    PARAMETERS = {"integrated_gradients_attribution": {"step_count": 50}}
elif XAI == "xrai":
    PARAMETERS = {"xrai_attribution": {"step_count": 50}}

parameters = aip.ExplanationParameters(PARAMETERS)

#### 解释元数据

让我们先深入了解解释元数据，它包括：

- `outputs`: 输出的标量值 -- 要解释的内容。例如，在分类问题中，输出为\[0.1, 0.2, 0.7\]，我们想要解释的是 0.7。考虑下面的公式，其中输出为 `y`，这就是我们想要解释的内容。

    y = f(x)

考虑下面的公式，其中输出为 `y` 和 `z`。由于我们只能对一个标量值进行属权，我们必须选择是要解释输出 `y` 还是 `z`。假设在这个例子中，模型是目标检测，`y` 和 `z` 分别代表边界框和物体分类。你需要选择解释哪一个输出。

    y, z = f(x)

`outputs` 的字典格式为:

    { "outputs": { "[your_display_name]":
                   "output_tensor_name": [layer]
                 }
    }

<blockquote>
 -  [your_display_name]: 你为要解释的输出分配的易读名称。一个常见的例子是 "probability"（概率）。<br/>
 -  "output_tensor_name": 用于标识要解释的输出层的键/值字段。<br/>
 -  [layer]: 要解释的输出层。在单任务模型中，比如表格回归器，这是模型中最后（最顶层）的层。
</blockquote>

- `inputs`: 影响输出的特征 -- 它们如何对结果做出贡献。考虑下面的公式，其中 `a` 和 `b` 是特征。我们需要选择要解释如何对结果有影响的特征。假设这个模型部署用于 A/B 测试，`a` 是用于预测的数据项，而 `b` 用于识别模型实例是A还是B。你需要选择 `a`（或其部分子集）作为特征，而不是 `b`，因为它对预测没有影响。

    y = f(a,b)

`inputs` 的最小字典格式为:

    { "inputs": { "[your_display_name]":
                  "input_tensor_name": [layer]
                 }
    }

<blockquote>
 -  [your_display_name]: 你为要解释的输入分配的易读名称。一个常见的例子是 "features"（特征）。<br/>
 -  "input_tensor_name": 用于标识特征属性的输入层的键/值字段。<br/>
 -  [layer]: 用于特征属性的输入层。在单输入张量模型中，这是模型中第一个（最底层）的层。
</blockquote>

由于模型的输入是表格形式的，你可以指定以下两个额外字段来作为报告/可视化的辅助功能:

<blockquote>
 - "modality": "image": 表示字段值为图像数据。
</blockquote>

由于模型的输入是图像，你可以指定以下额外字段作为报告/可视化的辅助功能:

<blockquote>
 - "modality": "image": 表示字段值为图像数据。
</blockquote>

In [ ]:
random_baseline = np.random.rand(32, 32, 3)
input_baselines = [{"number_vaue": x} for x in random_baseline]

INPUT_METADATA = {"input_tensor_name": CONCRETE_INPUT, "modality": "image"}

OUTPUT_METADATA = {"output_tensor_name": serving_output}

input_metadata = aip.ExplanationMetadata.InputMetadata(INPUT_METADATA)
output_metadata = aip.ExplanationMetadata.OutputMetadata(OUTPUT_METADATA)

metadata = aip.ExplanationMetadata(
    inputs={"image": input_metadata}, outputs={"class": output_metadata}
)

explanation_spec = aip.ExplanationSpec(metadata=metadata, parameters=parameters)

### 上传模型

使用这个辅助函数 `upload_model` 来上传您的模型，以 SavedModel 格式存储，到 `Model` 服务，该服务将为您的模型实例化一个 Vertex `Model` 资源实例。完成后，您可以像任何其他 Vertex `Model` 资源实例一样使用这个 `Model` 资源实例，比如部署到用于提供预测的 `Endpoint` 资源。

该辅助函数接受以下参数：

- `display_name`：`Endpoint` 服务的可读名称。
- `image_uri`：模型部署的容器镜像。
- `model_uri`：我们 SavedModel 文物的 Cloud Storage 路径。对于本教程，这是 `trainer/task.py` 保存模型文物的 Cloud Storage 位置，在变量 `MODEL_DIR` 中指定。

该辅助函数调用 `Model` 客户端服务的方法 `upload_model`，该方法接受以下参数：

- `parent`：`Dataset`、`Model` 和 `Endpoint` 资源的 Vertex 根路径。
- `model`：Vertex `Model` 资源实例的规范。

现在让我们深入了解 Vertex 模型规范 `model`。这是一个包含以下字段的字典对象：

- `display_name`：`Model` 资源的可读名称。
- `metadata_schema_uri`：由于您的模型是没有 Vertex `Dataset` 资源构建的，因此将保留为空 (`''`)。
- `artificat_uri`：SavedModel 格式模型存储在 Cloud Storage 中的路径。
- `container_spec`：这是将安装在 `Endpoint` 资源上的 Docker 容器的规范，`Model` 资源将从中提供预测。使用之前设置的变量 `DEPLOY_GPU != None` 来使用 GPU；否则只分配 CPU。
- `explanation_spec`：这是启用您的模型可解释性的规范。

将模型上传到 Vertex Model 资源会返回一个长时间运行的操作，因为可能需要一些时间。您可以调用 response.result()，这是一个同步调用，当 Vertex Model 资源准备就绪时将返回。

辅助函数返回对应的 Vertex Model 实例 upload_model_response.model 的完全限定标识符。您将保存这个标识符到后续步骤中的变量 model_to_deploy_id 中。

In [ ]:
IMAGE_URI = DEPLOY_IMAGE


def upload_model(display_name, image_uri, model_uri):

    model = aip.Model(
        display_name=display_name,
        artifact_uri=model_uri,
        metadata_schema_uri="",
        explanation_spec=explanation_spec,
        container_spec={"image_uri": image_uri},
    )

    response = clients["model"].upload_model(parent=PARENT, model=model)
    print("Long running operation:", response.operation.name)
    upload_model_response = response.result(timeout=180)
    print("upload_model_response")
    print(" model:", upload_model_response.model)
    return upload_model_response.model


model_to_deploy_id = upload_model(
    "cifar10-" + TIMESTAMP, IMAGE_URI, model_path_to_deploy
)

### 获取`Model`资源信息

现在让我们获取只有你的模型的模型信息。使用这个辅助函数`get_model`，并使用以下参数：

- `name`: `Model`资源的Vertex唯一标识符。

这个辅助函数调用了Vertex `Model`客户服务的方法`get_model`，并使用以下参数：

- `name`: `Model`资源的Vertex唯一标识符。

In [ ]:
def get_model(name):
    response = clients["model"].get_model(name=name)
    print(response)


get_model(model_to_deploy_id)

## 批量预测的模型部署

现在部署您为批量预测创建的训练过的Vertex `Model`资源。这与为按需预测部署`Model`资源有所不同。

对于在线预测，您需要：

1. 创建一个`Endpoint`资源来部署`Model`资源。

2. 将`Model`资源部署到`Endpoint`资源。

3. 向`Endpoint`资源发出在线预测请求。

对于批量预测，您需要：

1. 创建一个批量预测作业。

2. 作业服务将为批量预测请求提供资源。

3. 将批量预测请求的结果返回给调用者。

4. 作业服务将取消为批量预测请求提供的资源。

## 发出批量预测请求

现在对您部署的模型进行批量预测。

获取测试项

您将使用数据集的测试（保留）部分中的示例作为测试项。

In [ ]:
test_image_1 = x_test[0]
test_label_1 = y_test[0]
test_image_2 = x_test[1]
test_label_2 = y_test[1]
print(test_image_1.shape)

准备请求内容
您将发送CIFAR10图像作为压缩的JPG图像，而不是原始未压缩的字节：

- `cv2.imwrite`：使用openCV将未压缩的图像写入磁盘，以压缩的JPEG图像形式。
 - 将图像数据从\[0,1)范围反归一化为\[0,255)。
 - 将32位浮点值转换为8位无符号整数。

In [ ]:
import cv2

cv2.imwrite("tmp1.jpg", (test_image_1 * 255).astype(np.uint8))
cv2.imwrite("tmp2.jpg", (test_image_2 * 255).astype(np.uint8))

### 复制测试项目

对于批量预测，您需要将测试项目复制到您的云存储桶中。

In [ ]:
! gsutil cp tmp1.jpg $BUCKET_NAME/tmp1.jpg
! gsutil cp tmp2.jpg $BUCKET_NAME/tmp2.jpg

test_item_1 = BUCKET_NAME + "/tmp1.jpg"
test_item_2 = BUCKET_NAME + "/tmp2.jpg"

制作批量输入文件

现在制作一个批量输入文件，您将存储在本地云存储桶中。批量输入文件只能采用JSONL格式。对于JSONL文件，您为每个数据项（实例）制作一条字典条目。

字典包含以下键值对：

- `input_name`：基础模型的输入层名称。
- `'b64'`：指示内容为base64编码的键。
- `content`：经过压缩的JPG图像字节，作为base64编码的字符串。

预测请求中的每个实例是形式为的字典条目：

{ serving_input: {'b64': content} }

为了将图像数据传递到预测服务，您需要将字节编码为base64 - 这样在通过网络传输二进制数据时，内容就会受到保护，不易被修改。

- `tf.io.read_file`：将压缩的JPG图像作为原始字节读入内存。
- `base64.b64encode`：将原始字节编码为base64编码的字符串。

In [ ]:
import base64
import json

gcs_input_uri = BUCKET_NAME + "/" + "test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    bytes = tf.io.read_file(test_item_1)
    b64str = base64.b64encode(bytes.numpy()).decode("utf-8")
    data = {serving_input: {"b64": b64str}}
    f.write(json.dumps(data) + "\n")
    bytes = tf.io.read_file(test_item_2)
    b64str = base64.b64encode(bytes.numpy()).decode("utf-8")
    data = {serving_input: {"b64": b64str}}
    f.write(json.dumps(data) + "\n")

### 计算实例扩展

处理批量预测请求时，您有几种计算实例扩展选择：

- 单个实例：批量预测请求在单个计算实例上处理。
  - 将计算实例的最小(`MIN_NODES`)和最大(`MAX_NODES`)数量设置为一个。

- 手动扩展：批量预测请求在您手动指定的固定数量的计算实例上进行分割。
  - 将计算实例的最小(`MIN_NODES`)和最大(`MAX_NODES`)数量设置为相同数量的节点。当模型首次部署到实例时，固定数量的计算实例被配置，并且批量预测请求均匀分布在它们之间。

- 自动扩展：批量预测请求在可扩展数量的计算实例上分割。
  -设置计算实例的最小(`MIN_NODES`)数量为模型首次部署和取消部署时要配置的数量，并根据负载条件设置计算实例的最大(`MAX_NODES`)数量。

计算实例的最小数量对应于字段`min_replica_count`，最大计算实例的数量对应于字段`max_replica_count`，在您的后续部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 发起批量预测请求

现在您的两个测试项的批处理已经准备好了，让我们进行批处理请求。使用以下参数调用这个辅助函数 `create_batch_prediction_job`：

- `display_name`：预测任务的人类可读名称。
- `model_name`：`Model` 资源的 Vertex 完全限定标识符。
- `gcs_source_uri`：输入文件的 Cloud Storage 路径，即您之前创建的路径。
- `gcs_destination_output_uri_prefix`：服务将写入预测结果的 Cloud Storage 路径。
- `parameters`：用于提供预测结果的额外过滤参数。

辅助函数调用作业客户端服务的 `create_batch_prediction_job` 方法，使用以下参数：

- `parent`：Dataset、Model 和 Pipeline 资源的 Vertex 地点根路径。
- `batch_prediction_job`：批处理预测作业的规范。

现在让我们深入了解 `batch_prediction_job` 的规范：

- `display_name`：预测批处理作业的人类可读名称。
- `model`：`Model` 资源的 Vertex 完全限定标识符。
- `dedicated_resources`：用于批处理预测作业的计算资源。
  - `machine_spec`：要配置的计算实例。使用您之前设置的变量 `DEPLOY_GPU != None` 以使用 GPU；否则只分配 CPU。
  - `starting_replica_count`：最初要配置的计算实例数，之前您已将其设置为变量 `MIN_NODES`。
  - `max_replica_count`：要扩展到的最大计算实例数，之前您已将其设置为变量 `MAX_NODES`。
- `model_parameters`：用于提供预测结果的额外过滤参数。不支持自定义模型的额外参数。
- `input_config`：要预测实例的输入源和格式类型。
 - `instances_format`：批处理预测请求文件的格式：`csv` 或 `jsonl`。
 - `gcs_source`：一个或多个 Cloud Storage 路径列表，用于批处理预测请求。
- `output_config`：预测结果的输出目的地和格式。
 - `prediction_format`：批处理预测响应文件的格式：`csv` 或 `jsonl`。
 - `gcs_destination`：预测结果的输出目的地。

这个调用是一个异步操作。您将从响应对象中打印出一些选择的字段，包括：

- `name`：分配给批处理预测作业的 Vertex 完全限定标识符。
- `display_name`：预测批处理作业的人类可读名称。
- `model`：Model 资源的 Vertex 完全限定标识符。
- `generate_explanations`：预测结果是否提供了详细解释（可解释性）。
- `state`：预测作业的状态（等待中、运行中等）。

由于这个调用需要一些时间来执行，您可能会在 `state` 中得到 `JobState.JOB_STATE_PENDING`。

In [ ]:
BATCH_MODEL = "cifar10_batch-" + TIMESTAMP


def create_batch_prediction_job(
    display_name,
    model_name,
    gcs_source_uri,
    gcs_destination_output_uri_prefix,
    parameters=None,
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    batch_prediction_job = {
        "display_name": display_name,
        # Format: 'projects/{project}/locations/{location}/models/{model_id}'
        "model": model_name,
        "model_parameters": json_format.ParseDict(parameters, Value()),
        "input_config": {
            "instances_format": IN_FORMAT,
            "gcs_source": {"uris": [gcs_source_uri]},
        },
        "output_config": {
            "predictions_format": OUT_FORMAT,
            "gcs_destination": {"output_uri_prefix": gcs_destination_output_uri_prefix},
        },
        "dedicated_resources": {
            "machine_spec": machine_spec,
            "starting_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
        },
        "generate_explanation": True,
    }
    response = clients["job"].create_batch_prediction_job(
        parent=PARENT, batch_prediction_job=batch_prediction_job
    )
    print("response")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" model:", response.model)
    try:
        print(" generate_explanation:", response.generate_explanation)
    except:
        pass
    print(" state:", response.state)
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", response.labels)
    return response


IN_FORMAT = "jsonl"
OUT_FORMAT = "jsonl"

response = create_batch_prediction_job(
    BATCH_MODEL, model_to_deploy_id, gcs_input_uri, BUCKET_NAME
)

现在获取您创建的批量预测作业的唯一标识符。

In [ ]:
# The full unique ID for the batch job
batch_job_id = response.name
# The short numeric ID for the batch job
batch_job_short_id = batch_job_id.split("/")[-1]

print(batch_job_id)

### 获取有关批量预测作业的信息

使用此辅助函数`get_batch_prediction_job`，并传入以下参数：

- `job_name`：批量预测作业的 Vertex 完全限定标识符。

辅助函数调用作业客户端服务的`get_batch_prediction_job`方法，并使用以下参数：

- `name`：批量预测作业的 Vertex 完全限定标识符。在本教程中，您将向其传递批量预测作业的 Vertex 完全限定标识符 -- `batch_job_id`

辅助函数将返回存储预测结果的 Cloud Storage 路径 -- `gcs_destination`。

In [ ]:
def get_batch_prediction_job(job_name, silent=False):
    response = clients["job"].get_batch_prediction_job(name=job_name)
    if silent:
        return response.output_config.gcs_destination.output_uri_prefix, response.state

    print("response")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" model:", response.model)
    try:  # not all data types support explanations
        print(" generate_explanation:", response.generate_explanation)
    except:
        pass
    print(" state:", response.state)
    print(" error:", response.error)
    gcs_destination = response.output_config.gcs_destination
    print(" gcs_destination")
    print("  output_uri_prefix:", gcs_destination.output_uri_prefix)
    return gcs_destination.output_uri_prefix, response.state


predictions, state = get_batch_prediction_job(batch_job_id)

### 获取预测结果

当批量预测处理完成时，作业状态将为`JOB_STATE_SUCCEEDED`。

最后，请查看您设置为输出的云存储路径中存储的预测结果。预测结果将以JSONL格式显示，这是您在创建批量预测作业时指定的，在以名称`prediction`开头的子文件夹中，该文件夹下将有一个名为`prediction.results-xxxxx-of-xxxxx`的文件。

现在显示（cat）内容。您将看到多个JSON对象，每个对象代表一个预测结果。

最后，请查看您设置为输出的云存储路径中存储的解释。解释结果将以JSONL格式显示，这是您在创建批量解释作业时指定的，在以名称`prediction`开头的子文件夹中，该文件夹下将有一个名为`explanation-results-xxxx-of-xxxxx`的文件。

让我们显示（cat）内容。对于每个预测，您将看到一行数据 -- 在这种情况下，只有一行。该行是对应CIFAR10类别的softmax概率分布。

In [ ]:
def get_latest_predictions(gcs_out_dir):
    """ Get the latest prediction subfolder using the timestamp in the subfolder name"""
    folders = !gsutil ls $gcs_out_dir
    latest = ""
    for folder in folders:
        subfolder = folder.split("/")[-2]
        if subfolder.startswith("prediction-"):
            if subfolder > latest:
                latest = folder[:-1]
    return latest


while True:
    predictions, state = get_batch_prediction_job(batch_job_id, True)
    if state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("The job has not completed:", state)
        if state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        folder = get_latest_predictions(predictions)
        ! gsutil ls $folder/explanation.results*

        print("Results:")
        ! gsutil cat $folder/explanation.results*

        print("Errors:")
        ! gsutil cat $folder/prediction.errors*
        break
    time.sleep(60)

清理工作

要清理这个项目中使用的所有GCP资源，您可以[删除用于本教程的GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 端点
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME